In [59]:
from typing import Tuple

import jax 
import jax.numpy as jnp 

from craftax.craftax_env import CraftaxSymbolicEnv
from craftax.craftax.craftax_state import EnvParams, EnvState 

In [60]:
def cal_mem(shape:Tuple, dtype:int=32): 
    num_bytes = {
        32:4,
        16:2,
    }

    array = jnp.array(shape, dtype=jnp.float32)
    #print(array)
    prod = jnp.prod(array)
    print(prod)
    print(f"Number of bytes: {prod*num_bytes[dtype]}")
    return f"{prod*num_bytes[dtype] / 1e09} GB" 

In [62]:
env = CraftaxSymbolicEnv()
obs_space = env.observation_space(EnvParams)
obs = obs_space.sample(jax.random.PRNGKey(1))
obs.shape 

(8268,)

In [64]:
shape1 = (1, 1024, 4096,) + obs.shape
mem1 = cal_mem(shape1,)
print(mem1)

34678505000.0
Number of bytes: 138714021888.0
138.71401977539062 GB


In [47]:
1*1024*4096*1345

5641338880

In [51]:
1 * 4096 * 1024 * 8000 * 4 / 1e09

134.217728